In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import math

In [2]:
summarized = pd.read_csv("./results/summarized/1734322688_models_train_test_400.csv", usecols=['id', 'level', 'splitter', 'start_time', 'end_time'])
summarized = summarized[['id','level', 'splitter', 'start_time', 'end_time']]

summarized

times = pd.read_csv("./results/times/1734322688_400_SimplestCNNClassifier_8layers_Residual_times.csv")
# times = times[['level', 'splitter', 'start_time', 'end_time']]

# summarized.join(times, lsuffix="_l", rsuffix="_r")
times = pd.merge(summarized, times, on=["level","splitter"], suffixes=["_summarized","_times"])
times

,id_summarized,level,splitter,start_time_summarized,end_time_summarized,Unnamed: 0,experiment_id,id_times,model,batch_size,reserved_memory,acc,start_time_times,pred_time,end_time_times
0,0,class,prop_0-1/min_5/RandomSplit_0,1.734323e+09,1.734328e+09,0,1734322688,0,SimplestCNNClassifier_8layers_Residual,15000,6848.0,0.989240,1.738777e+09,1.738777e+09,1.738777e+09
1,1,class,prop_0-1/min_5/RandomSplit_14,1.734329e+09,1.734334e+09,1,1734322688,1,SimplestCNNClassifier_8layers_Residual,15000,7058.0,0.989726,1.738777e+09,1.738777e+09,1.738777e+09
2,2,class,prop_0-1/min_5/RandomSplit_56,1.734334e+09,1.734340e+09,2,1734322688,2,SimplestCNNClassifier_8layers_Residual,15000,7058.0,0.990454,1.738777e+09,1.738777e+09,1.738777e+09
3,3,class,prop_0-1/min_5/RandomSplit_84,1.734340e+09,1.734346e+09,3,1734322688,3,SimplestCNNClassifier_8layers_Residual,15000,7058.0,0.992072,1.738777e+09,1.738777e+09,1.738777e+09
4,4,class,prop_0-1/min_5/RandomSplit_92,1.734346e+09,1.734352e+09,4,1734322688,4,SimplestCNNClassifier_8layers_Residual,15000,7058.0,0.991506,1.738777e+09,1.738777e+09,1.738777e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,species,prop_0-05/min_10/StratifiedSplit2_84,1.735814e+09,1.735815e+09,395,1734322688,395,SimplestCNNClassifier_8layers_Residual,15000,506.0,0.948630,1.738787e+09,1.738787e+09,1.738787e+09
396,396,species,prop_0-05/min_10/StratifiedSplit2_92,1.735815e+09,1.735816e+09,396,1734322688,396,SimplestCNNClassifier_8layers_Residual,15000,506.0,0.933219,1.738787e+09,1.738787e+09,1.738787e+09
397,397,species,prop_0-05/min_10/StratifiedSplit2_101,1.735816e+09,1.735817e+09,397,1734322688,397,SimplestCNNClassifier_8layers_Residual,15000,506.0,0.943493,1.738787e+09,1.738787e+09,1.738787e+09
398,398,species,prop_0-05/min_10/StratifiedSplit2_105,1.735817e+09,1.735818e+09,398,1734322688,398,SimplestCNNClassifier_8layers_Residual,15000,506.0,0.950342,1.738787e+09,1.738787e+09,1.738787e+09


In [3]:
# times = pd.read_csv("./results/times_01/1734322688_80_SimplestCNNClassifier_8layers_Residual_times.csv")

times[["Prop", "Min", "Splitter_Seed"]] = times["splitter"].str.split("/", expand=True)
times

times[["Splitter","Seed"]] = times["Splitter_Seed"].str.split("_", expand=True)
times

times = times.drop(index = times.loc[(times["Prop"] == "prop_0-05") & (times["Min"] == "min_5")].index)
times

times["Train_Time"] = times["end_time_summarized"] - times["start_time_summarized"]
times["Prediction_Time"] = times["end_time_times"] - times["start_time_times"]

times[['experiment_id', 'level', 'acc', 'Prop', 'Min', 'Splitter', "Train_Time", 'Prediction_Time']]

,experiment_id,level,acc,Prop,Min,Splitter,Train_Time,Prediction_Time
0,1734322688,class,0.989240,prop_0-1,min_5,RandomSplit,5600.479617,22.607065
1,1734322688,class,0.989726,prop_0-1,min_5,RandomSplit,5684.736922,18.142404
2,1734322688,class,0.990454,prop_0-1,min_5,RandomSplit,5666.545494,17.963387
3,1734322688,class,0.992072,prop_0-1,min_5,RandomSplit,5727.036316,17.908510
4,1734322688,class,0.991506,prop_0-1,min_5,RandomSplit,5826.750788,17.949665
...,...,...,...,...,...,...,...,...
395,1734322688,species,0.948630,prop_0-05,min_10,StratifiedSplit2,564.176148,8.627599
396,1734322688,species,0.933219,prop_0-05,min_10,StratifiedSplit2,596.206263,8.633092
397,1734322688,species,0.943493,prop_0-05,min_10,StratifiedSplit2,578.412562,8.700821
398,1734322688,species,0.950342,prop_0-05,min_10,StratifiedSplit2,574.200165,8.809057


In [4]:
times = times.groupby(by=["level", "Prop"]).agg({
    "Train_Time":["min","mean","max"],
    "Prediction_Time":["min","mean","max"],
}).reset_index()

times

level       Prop   Train_Time                           Prediction_Time  \
                                min         mean          max             min   
0     class  prop_0-05  6090.422589  6132.281750  6210.519334       12.397831   
1     class   prop_0-1  5600.479617  5844.456780  6018.458421       17.908510   
2     class   prop_0-2  5522.091702  5582.913504  5719.413998       37.401426   
3    family  prop_0-05  4138.583620  4284.789259  4382.123646       11.831088   
4    family   prop_0-1  3968.350159  4036.371486  4161.215497       15.576366   
5    family   prop_0-2  3763.221136  3811.782233  3903.859997       30.946068   
6     genus  prop_0-05  2163.559673  2204.183419  2322.586681       10.527872   
7     genus   prop_0-1  2124.937845  2366.123288  2645.973104       12.467503   
8     genus   prop_0-2  2073.967122  2284.414085  2529.507148       16.290752   
9     order  prop_0-05  4974.336041  5020.216761  5168.253980       12.425622   
10    order   prop_0-1  4853.554132  4948.193886  5155.593368       17.075902   
11    order   prop_0-2  4554.186131  4599.272276  4705.527924       34.026916   
12  species  prop_0-05   554.746582   575.795515   605.567750        8.556786   
13  species   prop_0-1   542.555130   745.881234   947.079505        9.105844   
14  species   prop_0-2   531.049492   718.439598   921.535824       10.133633   

                          
         mean        max  
0   13.865637  17.873005  
1   19.155828  23.846806  
2   38.521782  43.046252  
3   12.185125  15.942960  
4   16.288856  20.651832  
5   31.384703  34.463496  
6   11.134925  14.427016  
7   13.872723  17.961772  
8   17.981926  22.846514  
9   13.064788  16.742444  
10  17.933116  22.324562  
11  34.361149  35.542452  
12   9.045734  11.931848  
13  10.310164  14.451753  
14  11.713841  15.938423

In [5]:
times["Train_Time_min"] = times.loc[:,("Train_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_max"] = times.loc[:,("Train_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_mean"] = times.loc[:,("Train_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times["Prediction_Time_min"] = times.loc[:,("Prediction_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Prediction_Time_max"] = times.loc[:,("Prediction_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Prediction_Time_mean"] = times.loc[:,("Prediction_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times = times.sort_values(by=[("Prediction_Time", "mean")], ascending=False)[["level","Prop","Train_Time_min","Train_Time_max","Train_Time_mean","Prediction_Time_min","Prediction_Time_max","Prediction_Time_mean"]]

In [6]:
times

,level,Prop,Train_Time_min,Train_Time_max,Train_Time_mean,Prediction_Time_min,Prediction_Time_max,Prediction_Time_mean
,,,,,,,,
2,class,prop_0-2,1:32:02,1:35:19,1:33:02,0:00:37,0:00:43,0:00:38
11,order,prop_0-2,1:15:54,1:18:25,1:16:39,0:00:34,0:00:35,0:00:34
5,family,prop_0-2,1:02:43,1:05:03,1:03:31,0:00:30,0:00:34,0:00:31
1,class,prop_0-1,1:33:20,1:40:18,1:37:24,0:00:17,0:00:23,0:00:19
8,genus,prop_0-2,0:34:33,0:42:09,0:38:04,0:00:16,0:00:22,0:00:17
10,order,prop_0-1,1:20:53,1:25:55,1:22:28,0:00:17,0:00:22,0:00:17
4,family,prop_0-1,1:06:08,1:09:21,1:07:16,0:00:15,0:00:20,0:00:16
7,genus,prop_0-1,0:35:24,0:44:05,0:39:26,0:00:12,0:00:17,0:00:13
0,class,prop_0-05,1:41:30,1:43:30,1:42:12,0:00:12,0:00:17,0:00:13
